In [25]:
#change original format
import re
import os
import pandas as pd

def change_format(path = r"data\excel_output\希望的格式.xlsx", outpath = r"data\excel_output\希望的格式1.xlsx"):
    out = pd.read_excel(path) #read excel

    for i in range (0,len(out)-1,2): #remove name row
        out.iloc[i:i+1,1:] = out.iloc[i+1:i+2,1:]
    out = out[out['Name'].notna()]

    out = out.rename(columns={'nad4L':'nad4l', 'tA':'trnA', 'tC':'trnC', 'tD':'trnD', 'tE':'trnE', #change column name
                            'tF':'trnF', 'tG':'trnG', 'tH':'trnH', 'tL1':'trnL1', 'tL2':'trnL2', 
                            'tM':'trnM', 'tN':'trnN', 'tP':'trnP', 'tQ':'trnQ', 'tR':'trnR', 
                            'tS1':'trnS1', 'tS2':'trnS2', 'tT':'trnT', 'tV':'trnV', 'tW':'trnW', 
                            'tY':'trnY'})

    out.to_excel (outpath, index = False, header=True)
change_format()

In [92]:
def file_to_df(path, dir = "data\\fasta_file\\"):
    data_dict = {}

    with open(dir + path,"r") as obj:
        data = obj.read()

    datalist = re.findall("[>][^>]+", data)

    for i in datalist:
        a = re.findall("_([a-zA-Z0-9.]{3,})", i)[-1]
        i = ''.join(i.splitlines()[1:])
        data_dict[a] = []
        data_dict[a].append(i)
        filename = re.match("(\w+)(\.)",path)
        df = pd.DataFrame(data_dict)
        df['Name'] = ">" + filename.group(1)
        df = df.sort_index(axis = 1)
    return df


In [79]:
def get_fasta_file_data(dir = "data\\fasta_file"):
    filelist = os.listdir(dir)
    templist = []
    for file in filelist:
        templist.append(file_to_df(file,dir+"\\"))
    return templist

In [91]:
def combine_data_frame(dir = "data\\fasta_file"):
    templist = get_fasta_file_data(dir)
    result = templist[0]
    n = 0
    while n < len(templist)-1:
        n += 1
        result = pd.concat([templist[n],result],ignore_index=True) #combine data and sort column name
    return result


In [88]:
def write_data_to_excel(excel_path,data): #add the new combined data to original excel
    original_data = pd.read_excel(excel_path, index_col=False)
    original_data = pd.concat([original_data,data],ignore_index=True)
    original_data.to_excel("data\excel_output\希望的格式1.xlsx") 

In [90]:
##main
original_excel_path = r"data\excel_output\希望的格式.xlsx"
formatted_excel_path = r"data\excel_output\output.xlsx"
fasta_file_dir = r"data\fasta_file"

##change format
change_format(original_excel_path,formatted_excel_path)
write_data_to_excel(formatted_excel_path,combine_data_frame(fasta_file_dir))
